## Data Description

The data consists of files obtained from different sources:

   - `contract.csv` — contract information
   - `personal.csv` — the client's personal data
   - `internet.csv` — information about Internet services
   - `phone.csv` — information about telephone services
    
In each file, the column `customerID` contains a unique code assigned to each client.

In [3]:
# Importing libraries
import pandas as pd
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
from functools import reduce
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
import sklearn.metrics as metrics
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.metrics import roc_curve, roc_auc_score
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [4]:
# reading datasets and store them into variables

# for loading files locally
try:
    df_contract = pd.read_csv('contract.csv')
    df_internet = pd.read_csv('internet.csv')
    df_personal = pd.read_csv('personal.csv')
    df_phone = pd.read_csv('phone.csv')
    
# for loading files on Practicum interface
except:
    df_contract = pd.read_csv('/datasets/final_provider/contract.csv')
    df_internet = pd.read_csv('/datasets/final_provider/internet.csv')
    df_personal = pd.read_csv('/datasets/final_provider/personal.csv')
    df_phone = pd.read_csv('/datasets/final_provider/phone.csv')

In [5]:
def inspect_dataframe(df, n_rows=3):
    print(df.info())
    display(df.sample(n_rows))

In [7]:
dataframes = [
    ("Contract", df_contract),
    ("Internet", df_internet),
    ("Personal", df_personal),
    ("Phone", df_phone)
]

for df_name, df in dataframes:
    print(f"{'-' * 30}\n{df_name}\n{'-' * 30}")
    inspect_dataframe(df)
    print("\n")

------------------------------
Contract
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB
None


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
6531,4923-ADWXJ,2018-01-01,No,Month-to-month,Yes,Bank transfer (automatic),65.80,1679.65
2199,8532-UEFWH,2015-10-01,No,Two year,No,Mailed check,25.75,1345.85
1828,1839-UMACK,2016-08-01,No,One year,No,Electronic check,25.05,949.85




------------------------------
Internet
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB
None


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
4748,3063-QFSZL,DSL,No,No,No,No,No,Yes
4291,6060-DRTNL,Fiber optic,No,No,Yes,No,Yes,No
3220,0746-JTRFU,DSL,No,No,No,No,No,No




------------------------------
Personal
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB
None


,customerID,gender,SeniorCitizen,Partner,Dependents
847,2316-ESMLS,Female,0,Yes,Yes
5217,5018-HEKFO,Female,0,No,No
5320,5338-YHWYT,Male,0,No,Yes




------------------------------
Phone
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB
None


,customerID,MultipleLines
2051,0866-QLSIR,Yes
5664,9933-QRGTX,No
1175,8390-FESFV,Yes


- No se tiene datos vacios o nulos

In [8]:
#Crearemmos una lista con todos los datos
dataframes_list = [df_contract, df_internet, df_personal, df_phone]

# Todos las tablas tienen 'customerID' y usando el join outer se uniran todas las tablas
merged_data = reduce(lambda left, right: pd.merge(left, right, on=['customerID'], how='outer'), dataframes_list)

In [9]:
inspect_dataframe(merged_data)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
 8   InternetService   5517 non-null   object 
 9   OnlineSecurity    5517 non-null   object 
 10  OnlineBackup      5517 non-null   object 
 11  DeviceProtection  5517 non-null   object 
 12  TechSupport       5517 non-null   object 
 13  StreamingTV       5517 non-null   object 
 14  StreamingMovies   5517 non-null   object 
 15  gender            7043 non-null   object 
 16  SeniorCitizen     7043 non-null   int64  


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,gender,SeniorCitizen,Partner,Dependents,MultipleLines
5121,4903-CNOZC,2017-09-01,No,One year,No,Credit card (automatic),70.90,1964.6,DSL,No,Yes,Yes,No,No,Yes,Male,0,No,No,Yes
1857,9359-JANWS,2014-09-01,No,Two year,No,Credit card (automatic),58.90,3857.1,DSL,Yes,No,Yes,Yes,No,No,Female,0,Yes,No,No
1133,3156-QLHBO,2019-12-01,No,Month-to-month,No,Mailed check,19.25,48.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male,0,No,Yes,No


- Se entiende que los servicios customerID que tienen null o no tienen datos es porque son clientes sin esos servicios. Lo ideal seria completarlo con 'No'

In [10]:
df_nonull = merged_data.fillna('No')
df_nonull.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  gender            7043 non-null   object 
 16  SeniorCitizen     7043 non-null   int64  


In [11]:
df_nonull.sample(5)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,gender,SeniorCitizen,Partner,Dependents,MultipleLines
4850,0931-MHTEM,2015-08-01,No,One year,Yes,Credit card (automatic),100.05,5299.65,Fiber optic,Yes,Yes,No,No,Yes,Yes,Female,0,No,No,No
1916,4720-VSTSI,2016-02-01,2019-10-01 00:00:00,Month-to-month,Yes,Bank transfer (automatic),84.80,3862.55,Fiber optic,No,Yes,Yes,No,No,No,Female,0,No,No,Yes
2465,1740-CSDJP,2019-12-01,2020-01-01 00:00:00,Month-to-month,Yes,Bank transfer (automatic),35.25,35.25,DSL,No,No,No,No,No,Yes,Male,0,No,No,No
2745,4826-XTSOH,2019-10-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,86.05,86.05,Fiber optic,No,No,No,No,Yes,No,Male,1,Yes,No,Yes
5733,4817-KEQSP,2014-03-01,No,Two year,No,Bank transfer (automatic),19.85,1326.35,No,No,No,No,No,No,No,Female,0,Yes,Yes,No
